In [ ]:
import copy
import logging

import numpy as np

from atmodeller import (
    InteriorAtmosphere,
    Planet,
    Species,
    SpeciesCollection,
    bulk_silicate_earth_abundances,
    debug_logger,
    earth_oceans_to_hydrogen_mass,
)
from atmodeller.solubility import get_solubility_models
from atmodeller.thermodata import IronWustiteBuffer

logger = debug_logger()
logger.setLevel(logging.INFO)
# For more output use DEBUG
# logger.setLevel(logging.DEBUG)

# Trappist 1-e models from Bower et al. (2025)

## High temperature atmospheric diversity (gas + C_cr)

Parameters for the simulations

In [ ]:
number_of_realisations = 10000
surface_temperature = 1800.0  # Must be float
mantle_melt_fraction = 0.1  # 0.1

# For no particular reason, use 24 as the random seed
RANDOM_SEED = 24

WRITE_OUTPUT = True

Parameter space to probe. Generate once to ensure the same values are used across the high temperature models allowing direct comparison.

In [ ]:
np.random.seed(RANDOM_SEED)
log10_number_oceans = np.random.uniform(-1, 1, number_of_realisations)
log10_ch_ratios = np.random.uniform(-1, 1, number_of_realisations)
fO2_log10_shifts = np.random.uniform(-5, 5, number_of_realisations)

h_kg = earth_oceans_to_hydrogen_mass(10**log10_number_oceans)
c_kg = h_kg * 10**log10_ch_ratios

trappist1e_mantle_mass = 2.912e24
trappist1e_planet_mass = trappist1e_mantle_mass / (1 - 0.295334691460966)
trappist1e_surface_radius = 5.861e6

fugacity_constraints = {"O2_g": IronWustiteBuffer(fO2_log10_shifts)}

Species that we need

In [ ]:
H2O_g = Species.create_gas("H2O_g")
H2_g = Species.create_gas("H2_g")
O2_g = Species.create_gas("O2_g")
CO_g = Species.create_gas("CO_g")
CO2_g = Species.create_gas("CO2_g")
CH4_g = Species.create_gas("CH4_g")
N2_g = Species.create_gas("N2_g")
NH3_g = Species.create_gas("NH3_g")
S2_g = Species.create_gas("S2_g")
H2S_g = Species.create_gas("H2S_g")
SO2_g = Species.create_gas("SO2_g")
SO_g = Species.create_gas("SO_g")
Cl2_g = Species.create_gas("Cl2_g")
# Graphite can also be present in high temperature atmospheres
C_cr = Species.create_condensed("C_cr")
# Condensates below are used for cool atmosphere cases
H2O_l = Species.create_condensed("H2O_l")
S_cr = Species.create_condensed("S_cr")

Species to consider, where solubility is not included

In [ ]:
species = SpeciesCollection(
    (
        H2_g,
        H2O_g,
        O2_g,
        CO_g,
        CO2_g,
        CH4_g,
        N2_g,
        NH3_g,
        S2_g,
        H2S_g,
        SO2_g,
        SO_g,
        Cl2_g,
        C_cr,
    )
)

TRAPPIST-1e planet properties

Mass and radius measurements from Agol et al. 2021; Mantle mass determined assuming same proportion as Earth

In [ ]:
trappist1e_1800K = Planet(
    surface_temperature=surface_temperature,
    planet_mass=trappist1e_planet_mass,
    surface_radius=trappist1e_surface_radius,
    mantle_melt_fraction=mantle_melt_fraction,
)

Earth planet properties, which are required to scale the bulk volatile inventories for Trappist-1e. Default parameters are Earth so we only need to specify the temperature.

In [ ]:
earth = Planet(surface_temperature=surface_temperature)

In [ ]:
earth_bse = bulk_silicate_earth_abundances()

Compute the reservoir sizes for TRAPPIST 1-e, assuming the same ppmw as Earth:

In [ ]:
trappist1e_bse = copy.deepcopy(earth_bse)
mass_scale_factor = trappist1e_1800K.mantle_mass / earth.mantle_mass

for element, values in trappist1e_bse.items():
    trappist1e_bse[element] = {key: value * mass_scale_factor for key, value in values.items()}  # type:ignore

trappist1e_bse

In [ ]:
interior_atmosphere_nosol = InteriorAtmosphere(species, mass_logarithmic_error=False)

mass_constraints = {
    "H": h_kg,
    "C": c_kg,
    "N": trappist1e_bse["N"]["mean"],
    "S": trappist1e_bse["S"]["mean"],
    "Cl": trappist1e_bse["Cl"]["mean"],
}

interior_atmosphere_nosol.solve(
    planet=trappist1e_1800K,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
)
output_nosol = interior_atmosphere_nosol.output

# Quick look at the solution
# solution = output_nosol.quick_look()

# Get complete solution as a dictionary
# solution_asdict = output.asdict()
# logger.info(solution_asdict)

if WRITE_OUTPUT:
    # Write the complete solution to Excel
    output_nosol.to_excel("t1e_1800K_nosol_C")

    # Write the data to a pickle file with dataframes
    output_nosol.to_pickle("t1e_1800K_nosol_C")

Cases with solubility

In [ ]:
solubility_models = get_solubility_models()

In [ ]:
H2O_g = Species.create_gas("H2O_g", solubility=solubility_models["H2O_basalt_dixon95"])
H2_g = Species.create_gas("H2_g", solubility=solubility_models["H2_basalt_hirschmann12"])
O2_g = Species.create_gas("O2_g")
CO_g = Species.create_gas("CO_g", solubility=solubility_models["CO_basalt_yoshioka19"])
CO2_g = Species.create_gas("CO2_g", solubility=solubility_models["CO2_basalt_dixon95"])
CH4_g = Species.create_gas("CH4_g", solubility=solubility_models["CH4_basalt_ardia13"])
N2_g = Species.create_gas("N2_g", solubility=solubility_models["N2_basalt_libourel03"])
NH3_g = Species.create_gas("NH3_g")
S2_g = Species.create_gas("S2_g", solubility=solubility_models["S2_basalt_boulliung23"])
H2S_g = Species.create_gas("H2S_g")
SO2_g = Species.create_gas("SO2_g")
SO_g = Species.create_gas("SO_g")
Cl2_g = Species.create_gas("Cl2_g", solubility=solubility_models["Cl2_basalt_thomas21"])
C_cr = Species.create_condensed("C_cr")

species = SpeciesCollection(
    (
        H2_g,
        H2O_g,
        O2_g,
        CO_g,
        CO2_g,
        CH4_g,
        N2_g,
        NH3_g,
        S2_g,
        H2S_g,
        SO2_g,
        SO_g,
        Cl2_g,
        C_cr,
    )
)

In [ ]:
interior_atmosphere_withsol = InteriorAtmosphere(species)

mass_constraints = {
    "H": h_kg,
    "C": c_kg,
    "N": trappist1e_bse["N"]["mean"],
    "S": trappist1e_bse["S"]["mean"],
    "Cl": trappist1e_bse["Cl"]["mean"],
}

interior_atmosphere_withsol.solve(
    planet=trappist1e_1800K,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
)
output_withsol = interior_atmosphere_withsol.output

# Quick look at the solution
# solution = output.quick_look()

# Get complete solution as a dictionary
# solution_asdict = output.asdict()
# logger.info(solution_asdict)

if WRITE_OUTPUT:
    # Write the complete solution to Excel
    output_withsol.to_excel("t1e_1800K_withsol_C")

    # Write the data to a pickle file with dataframes
    output_withsol.to_pickle("t1e_1800K_withsol_C")

# Cool high temperature atmospheres (gas + C_cr)

In [ ]:
final_surface_temperature = 280.0  # Must be a float
water_condensation = 600.0  # First thermo data for H2O(l)
sulfur_freezing = 380.0  # 388.36  # First thermo data for S(cr)

In [ ]:
interior_atmosphere_280K = InteriorAtmosphere(species)

trappist1e_280K = Planet(
    surface_temperature=final_surface_temperature,
    planet_mass=trappist1e_planet_mass,
    surface_radius=trappist1e_surface_radius,
    mantle_melt_fraction=0.0,  # Always zero because the planet is solidified
)

withsol_dict = output_withsol.asdict()

# Create a dictionary with the series
mass_constraints = {
    "H": withsol_dict["element_H"]["atmosphere_mass"],
    "S": withsol_dict["element_S"]["atmosphere_mass"],
    "N": withsol_dict["element_N"]["atmosphere_mass"],
    "O": withsol_dict["element_O"]["atmosphere_mass"],
    # For C, we need to add the condensed mass to the atmosphere mass since the assumption is that
    # they remain in equilibrium
    "C": withsol_dict["element_C"]["atmosphere_mass"]
    + withsol_dict["element_C"]["condensed_mass"],
    "Cl": withsol_dict["element_Cl"]["atmosphere_mass"],
}

interior_atmosphere_280K.solve(
    planet=trappist1e_280K,
    mass_constraints=mass_constraints,
)
output_280K = interior_atmosphere_280K.output

if WRITE_OUTPUT:
    # Write the complete solution to Excel
    output_280K.to_excel("t1e_280K_gas_C")

    # Write the data to a pickle file with dataframes
    output_280K.to_pickle("t1e_280K_gas_C")

## Cool atmospheres with condensates (gas + C_cr + H2O_l + S_cr)

In [ ]:
species = SpeciesCollection(
    (
        H2_g,
        H2O_g,
        O2_g,
        CO_g,
        CO2_g,
        CH4_g,
        N2_g,
        NH3_g,
        S2_g,
        H2S_g,
        SO2_g,
        SO_g,
        Cl2_g,
        C_cr,
        H2O_l,
        S_cr,
    )
)

interior_atmosphere_280K_with_condensates = InteriorAtmosphere(species)

trappist1e_280K_with_condensates = Planet(
    surface_temperature=final_surface_temperature,
    planet_mass=trappist1e_planet_mass,
    surface_radius=trappist1e_surface_radius,
    mantle_melt_fraction=0.0,  # Always zero because the planet is solidified
)

output_280K_dict = output_280K.asdict()

# Create a dictionary with the series
mass_constraints = {
    "H": output_280K_dict["element_H"]["total_mass"],
    "S": output_280K_dict["element_S"]["total_mass"],
    "N": output_280K_dict["element_N"]["total_mass"],
    "O": output_280K_dict["element_O"]["total_mass"],
    "C": output_280K_dict["element_C"]["total_mass"],
    "Cl": output_280K_dict["element_Cl"]["total_mass"],
}

interior_atmosphere_280K_with_condensates.solve(
    planet=trappist1e_280K,
    mass_constraints=mass_constraints,
)
output_280K_with_condensates = interior_atmosphere_280K_with_condensates.output

if WRITE_OUTPUT:
    # Write the complete solution to Excel
    output_280K_with_condensates.to_excel("t1e_280K_with_condensates")

    # Write the data to a pickle file with dataframes
    output_280K_with_condensates.to_pickle("t1e_280K_with_condensates")